In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
csv_data = pd.read_csv("/Users/andrea/Desktop/ECDS Y1S2/SC1015 Intro to Data Science and Artificial Intelligence/SC1015 Mini Project/Anime.csv")
csv_data.head()


,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
0,16498,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",TV,25.0,Finished Airing,"Apr 7, 2013",...,"Gore, Military, Survival",Shounen,24.0,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,3524109,155695
1,1535,Death Note,DN,デスノート,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37.0,Finished Airing,"Oct 4, 2006",...,Psychological,Shounen,23.0,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,3504535,159701
2,5114,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi Fullmetal Alchemist, ...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64.0,Finished Airing,"Apr 5, 2009",...,Military,Shounen,24.0,R - 17+ (violence & profanity),9.131,900398.0,12.0,3,2978455,207772
3,30276,One Punch Man,"One Punch-Man, One-Punch Man, OPM",ワンパンマン,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12.0,Finished Airing,"Oct 5, 2015",...,"Parody, Super Power",Seinen,24.0,R - 17+ (violence & profanity),8.511,19066.0,1112.0,4,2879907,59651
4,11757,Sword Art Online,"S.A.O, SAO",ソードアート・オンライン,Sword Art Online,Ever since the release of the innovative Nerve...,TV,25.0,Finished Airing,"Jul 8, 2012",...,"Love Polygon, Video Game",Unknown,23.0,PG-13 - Teens 13 or older,7.201,990254.0,29562.0,5,2813565,64997


In [3]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime as dt

csv_data = pd.read_csv("Anime.csv")
csv_data.head()

csv_data.dropna(subset=['Score'], inplace=True)
csv_data

csv_data.drop_duplicates()

newdf = pd.DataFrame(csv_data)
newdf["English"].fillna("Unknown", inplace=True)
for index, row in newdf.iterrows():
    # If the value in the "English" column is "Unknown"
    if row["English"] == "Unknown":
        # Fill in missing data from "Title" to "English"
        newdf.loc[index, "English"] = row["Title"]

newdf = csv_data
# 1.extract only the day of broadcast
newdf['Day'] = newdf['Broadcast'].apply(lambda x: x.split(' ')[0])
# 2. extract only the season of broadcast, ignore year
newdf['Season'] = newdf['Premiered'].apply(lambda x: x.split(' ')[0])
# 3. extract only the year of broadcast, ignore season, extract last 4 characters if value is not 'Unknown'
newdf["Start_Aired"] = pd.to_datetime(newdf["Start_Aired"], errors="coerce")
newdf["Year"] = newdf["Start_Aired"].dt.year

# 4. calculate total duration of each show
#Append Total Duration = Number of Episodes * Duration into the dataset.
results_list = []

for i, row in newdf.iterrows():
    result = (row['Episodes']) * row['Duration_Minutes']
    results_list.append(result)

newdf['Total duration'] = results_list

# 5. Calculate the ratio of Favourites vs Members for each show, round to 4dp
results_list = []

for i, row in newdf.iterrows():
    result = (row['Favorites']) / (row['Members'])
    #result = round(result, 4)
    results_list.append(result)

newdf['FMratio'] = results_list

# 6. remove anime with Status as not yet aired as they do not have a score
newdf = newdf[newdf["Status"].str.contains("Not yet aired") == False]
# 7. combine Themes and Genres to new column Themes_Genres
newdf["Themes_Genres"] = newdf["Themes"] + ", " + newdf["Genres"]
# 8. delete irrelevant columns
newdf = newdf.drop(["Premiered", "Broadcast", "Genres", "Themes", "Favorites", "Members", "Episodes", "Duration_Minutes"], axis=1)
newdf = newdf.drop(["Title", "ID","Japanese","Synonyms","Synopsis","Start_Aired","End_Aired"], axis=1)
# 9. Delete anime with Year greater than 2022
mask = newdf['Year'] > 2022
newdf = newdf.drop(newdf[mask].index)
# Rank unknown
newdf["Ranked"].fillna(value = "No Rank", inplace = True)

# Year unknown
newdf["Year"].fillna(value = "Unknown", inplace = True)

# Total duration unknown
newdf["Total duration"].fillna(value = "Unknown", inplace = True)

newdf

,English,Type,Status,Producers,Licensors,Studios,Source,Demographics,Rating,Score,Scored_Users,Ranked,Popularity,Day,Season,Year,Total duration,FMratio,Themes_Genres
0,Attack on Titan,TV,Finished Airing,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Wit Studio,Manga,Shounen,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,Sundays,Spring,2013.0,600.0,0.044180,"Gore, Military, Survival, Action, Drama"
1,Death Note,TV,Finished Airing,"VAP, Konami, Ashi Productions, Nippon Televisi...",VIZ Media,Madhouse,Manga,Shounen,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,Wednesdays,Fall,2006.0,851.0,0.045570,"Psychological, Supernatural, Suspense"
2,Fullmetal Alchemist Brotherhood,TV,Finished Airing,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones,Manga,Shounen,R - 17+ (violence & profanity),9.131,900398.0,12.0,3,Sundays,Spring,2009.0,1536.0,0.069758,"Military, Action, Adventure, Drama, Fantasy"
3,One Punch Man,TV,Finished Airing,"TV Tokyo, Bandai Visual, Lantis, Asatsu DK, Ba...",VIZ Media,Madhouse,Web manga,Seinen,R - 17+ (violence & profanity),8.511,19066.0,1112.0,4,Mondays,Fall,2015.0,288.0,0.020713,"Parody, Super Power, Action, Comedy"
4,Sword Art Online,TV,Finished Airing,"Aniplex, Genco, DAX Production, ASCII Media Wo...",Aniplex of America,A-1 Pictures,Light novel,Unknown,PG-13 - Teens 13 or older,7.201,990254.0,29562.0,5,Sundays,Summer,2012.0,575.0,0.023101,"Love Polygon, Video Game, Action, Adventure, F..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17720,O Light,Music,Finished Airing,Unknown,Unknown,Unknown,Original,Unknown,PG-13 - Teens 13 or older,6.231,108.0,78922.0,17746,Unknown,Unknown,2016.0,4.0,0.000000,"Music, Unknown"
17737,Mr.,Music,Finished Airing,Unknown,Unknown,Calf Studio,Original,Unknown,PG-13 - Teens 13 or older,6.541,110.0,61512.0,17763,Unknown,Unknown,2022.0,3.0,0.000000,"Music, Unknown"
17820,Greenlights Serenade,Music,Finished Airing,Unknown,Unknown,Unknown,Original,Unknown,G - All Ages,5.931,109.0,93762.0,17845,Unknown,Unknown,2018.0,4.0,0.005525,"Music, Unknown"
17857,Heart Beat (2019),Music,Finished Airing,"USAGI Production, INoNaKa Music",Unknown,Unknown,Original,Unknown,PG-13 - Teens 13 or older,5.921,103.0,94362.0,17883,Unknown,Unknown,2019.0,4.0,0.000000,"Idols (Female), Music, Unknown"


In [4]:
# Split the 'Themes_Genres' column by ',' and create dummy columns for each theme/genre
newdf = newdf.join(newdf['Themes_Genres'].str.get_dummies(', '))

# Drop the original 'Themes_Genres' column
newdf.drop(columns=['Themes_Genres'], inplace=True)

# Split the 'Producers' column by ',' and create dummy columns for each producer

# Create 'Producers' dummy variables
newdf_producers = newdf['Producers'].str.get_dummies(', ')
newdf_producers.columns = [f'Producer_{col}' for col in newdf_producers.columns]  # Add a prefix to column names
if 'Unknown' in newdf_producers.columns:
    newdf_producers['Producer_Unknown'] = newdf_producers['Unknown']
    newdf_producers.drop(columns=['Unknown'], inplace=True)
newdf = newdf.join(newdf_producers)

# Create 'Licensors' dummy variables
newdf_licensors = newdf['Licensors'].str.get_dummies(', ')
newdf_licensors.columns = [f'Licensor_{col}' for col in newdf_licensors.columns]  # Add a prefix to column names
if 'Unknown' in newdf_licensors.columns:
    newdf_licensors['Licensor_Unknown'] = newdf_licensors['Unknown']
    newdf_licensors.drop(columns=['Unknown'], inplace=True)
newdf = newdf.join(newdf_licensors)

# Create 'Studios' dummy variables
newdf_studios = newdf['Studios'].str.get_dummies(', ')
newdf_studios.columns = [f'Studio_{col}' for col in newdf_studios.columns]  # Add a prefix to column names
if 'Unknown' in newdf_studios.columns:
    newdf_studios['Studio_Unknown'] = newdf_studios['Unknown']
    newdf_studios.drop(columns=['Unknown'], inplace=True)
newdf = newdf.join(newdf_studios)

# Replace all occurrences of 1 in the 'Unknown' column with 0
newdf['Unknown'] = newdf['Unknown'].replace(1, 0)

newdf

,English,Type,Status,Producers,Licensors,Studios,Source,Demographics,Rating,Score,...,Studio_ixtl,Studio_l-a-unch・BOX,Studio_monofilmo,Studio_pH Studio,Studio_production doA,Studio_studio MOTHER,Studio_team Yamahitsuji,Studio_teamKG,Studio_ufotable,Studio_yell
0,Attack on Titan,TV,Finished Airing,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Wit Studio,Manga,Shounen,R - 17+ (violence & profanity),8.531,...,0,0,0,0,0,0,0,0,0,0
1,Death Note,TV,Finished Airing,"VAP, Konami, Ashi Productions, Nippon Televisi...",VIZ Media,Madhouse,Manga,Shounen,R - 17+ (violence & profanity),8.621,...,0,0,0,0,0,0,0,0,0,0
2,Fullmetal Alchemist Brotherhood,TV,Finished Airing,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones,Manga,Shounen,R - 17+ (violence & profanity),9.131,...,0,0,0,0,0,0,0,0,0,0
3,One Punch Man,TV,Finished Airing,"TV Tokyo, Bandai Visual, Lantis, Asatsu DK, Ba...",VIZ Media,Madhouse,Web manga,Seinen,R - 17+ (violence & profanity),8.511,...,0,0,0,0,0,0,0,0,0,0
4,Sword Art Online,TV,Finished Airing,"Aniplex, Genco, DAX Production, ASCII Media Wo...",Aniplex of America,A-1 Pictures,Light novel,Unknown,PG-13 - Teens 13 or older,7.201,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17720,O Light,Music,Finished Airing,Unknown,Unknown,Unknown,Original,Unknown,PG-13 - Teens 13 or older,6.231,...,0,0,0,0,0,0,0,0,0,0
17737,Mr.,Music,Finished Airing,Unknown,Unknown,Calf Studio,Original,Unknown,PG-13 - Teens 13 or older,6.541,...,0,0,0,0,0,0,0,0,0,0
17820,Greenlights Serenade,Music,Finished Airing,Unknown,Unknown,Unknown,Original,Unknown,G - All Ages,5.931,...,0,0,0,0,0,0,0,0,0,0
17857,Heart Beat (2019),Music,Finished Airing,"USAGI Production, INoNaKa Music",Unknown,Unknown,Original,Unknown,PG-13 - Teens 13 or older,5.921,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Find columns with float data type
float_columns = newdf.select_dtypes(include='float')
print("Columns with float data type:")
print(float_columns)

# Find columns with string data type
string_columns = newdf.select_dtypes(include='object')
print("Columns with string data type:")
print(string_columns)

Columns with float data type:
       Score  Scored_Users   FMratio
0      8.531      519803.0  0.044180
1      8.621      485487.0  0.045570
2      9.131      900398.0  0.069758
3      8.511       19066.0  0.020713
4      7.201      990254.0  0.023101
...      ...           ...       ...
17720  6.231         108.0  0.000000
17737  6.541         110.0  0.000000
17820  5.931         109.0  0.005525
17857  5.921         103.0  0.000000
17988  6.261         113.0  0.000000

[14556 rows x 3 columns]
Columns with string data type:
                               English   Type           Status  \
0                      Attack on Titan     TV  Finished Airing   
1                           Death Note     TV  Finished Airing   
2      Fullmetal Alchemist Brotherhood     TV  Finished Airing   
3                        One Punch Man     TV  Finished Airing   
4                     Sword Art Online     TV  Finished Airing   
...                                ...    ...              ...   
17720  

In [6]:
# ensure that FMratio is not 0
# scaling FMratio
scaling_factor = 10000 # appropriate scaling factor so there is a larger range in integer FMratio
newdf['FMratio'] = (newdf['FMratio'] * scaling_factor).round().astype('int64')

# Define the scaling factor
scaling_factor = 2  # appropriate scaling factor so that there is a larger range in integer scores

# Apply the scaling factor and round to nearest integer
newdf['Score'] = (newdf['Score'] * scaling_factor).round().astype('int64')

In [7]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert object columns to string
float_columns = newdf.select_dtypes(include='object')
newdf[float_columns.columns] = newdf[float_columns.columns].astype('string')


# Convert float columns to int64
float_columns = newdf.select_dtypes(include='float')
newdf[float_columns.columns] = newdf[float_columns.columns].astype('int64')

newdf

,English,Type,Status,Producers,Licensors,Studios,Source,Demographics,Rating,Score,...,Studio_ixtl,Studio_l-a-unch・BOX,Studio_monofilmo,Studio_pH Studio,Studio_production doA,Studio_studio MOTHER,Studio_team Yamahitsuji,Studio_teamKG,Studio_ufotable,Studio_yell
0,Attack on Titan,TV,Finished Airing,"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Wit Studio,Manga,Shounen,R - 17+ (violence & profanity),17,...,0,0,0,0,0,0,0,0,0,0
1,Death Note,TV,Finished Airing,"VAP, Konami, Ashi Productions, Nippon Televisi...",VIZ Media,Madhouse,Manga,Shounen,R - 17+ (violence & profanity),17,...,0,0,0,0,0,0,0,0,0,0
2,Fullmetal Alchemist Brotherhood,TV,Finished Airing,"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones,Manga,Shounen,R - 17+ (violence & profanity),18,...,0,0,0,0,0,0,0,0,0,0
3,One Punch Man,TV,Finished Airing,"TV Tokyo, Bandai Visual, Lantis, Asatsu DK, Ba...",VIZ Media,Madhouse,Web manga,Seinen,R - 17+ (violence & profanity),17,...,0,0,0,0,0,0,0,0,0,0
4,Sword Art Online,TV,Finished Airing,"Aniplex, Genco, DAX Production, ASCII Media Wo...",Aniplex of America,A-1 Pictures,Light novel,Unknown,PG-13 - Teens 13 or older,14,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17720,O Light,Music,Finished Airing,Unknown,Unknown,Unknown,Original,Unknown,PG-13 - Teens 13 or older,12,...,0,0,0,0,0,0,0,0,0,0
17737,Mr.,Music,Finished Airing,Unknown,Unknown,Calf Studio,Original,Unknown,PG-13 - Teens 13 or older,13,...,0,0,0,0,0,0,0,0,0,0
17820,Greenlights Serenade,Music,Finished Airing,Unknown,Unknown,Unknown,Original,Unknown,G - All Ages,12,...,0,0,0,0,0,0,0,0,0,0
17857,Heart Beat (2019),Music,Finished Airing,"USAGI Production, INoNaKa Music",Unknown,Unknown,Original,Unknown,PG-13 - Teens 13 or older,12,...,0,0,0,0,0,0,0,0,0,0


### finding accuracy of using all given variables to predict `Score`

In [8]:
# Instantiate the OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Encode categorical columns using ordinal encoding
X_encoded = ordinal_encoder.fit_transform(newdf.drop(['Score', 'English','Producers','Studios', 'Licensors'], axis=1))  # Update column names as a list

# Encode the 'Score' column using label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(newdf['Score'])

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=1)

# Instantiate the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Fit the model to the training data
rf_classifier.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_classifier.predict(X_test)

# Decode the predicted labels back to original score values
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.771978021978022


In [9]:
# initialize data
y = newdf['Score'].values.reshape(-1,)
X = newdf.drop(['Score'], axis=1)
cat_cols = newdf.select_dtypes(include=['object']).columns
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7278, 2390), (7278, 2390), (7278,), (7278,))

In [10]:
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error

# Encode categorical columns using ordinal encoding
X = ordinal_encoder.fit_transform(newdf.drop(['Score', 'English','Producers','Studios', 'Licensors'], axis=1))  # Update column names as a list

#y = ordinal_encoder.fit_transform(newdf['Score'])


y = newdf['Score'].values.reshape(-1,)
#X = newdf.drop(['Score'], axis=1)
cat_cols = newdf.select_dtypes(include=['object']).columns
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# initialize Pool
train_pool = Pool(X_train, 
                  y_train, 
                  cat_features=cat_cols_idx)
test_pool = Pool(X_test,
                 y_test,
                 cat_features=cat_cols_idx)
# specify the training parameters 
model = CatBoostRegressor(iterations=1100,
                          depth=5, 
                          learning_rate=0.05,
                          verbose=0,
                          loss_function='RMSE')
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
y_train_pred = model.predict(train_pool)
y_test_pred = model.predict(test_pool)

rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"RMSE score for train {round(rmse_train,3)} points, and for test {round(rmse_test,3)} points")


RMSE score for train 0.472 points, and for test 0.601 points


In [11]:
# Baseline scores (assuming the same prediction for all data samples)
rmse_bs_train = mean_squared_error(y_train, [np.mean(y_train)]*len(y_train), squared=False)
rmse_bs_test = mean_squared_error(y_test, [np.mean(y_train)]*len(y_test), squared=False)
print(f"RMSE baseline score for train {round(rmse_bs_train,3)} points, and for test {round(rmse_bs_test,3)} points")

RMSE baseline score for train 1.85 points, and for test 1.839 points
